In [1]:
from bs4 import BeautifulSoup
import urllib
import re 

In [2]:
def search(query):
    address = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(query))

    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()

    soup = BeautifulSoup(htmlResult)

    [s.extract() for s in soup('span')]
    unwantedTags = ['strong', 'cite']
    
    for tag in unwantedTags:
        for match in soup.findAll(tag):
            match.replaceWithChildren()

    results = soup.findAll('li', { "class" : "b_algo" })

    return results

In [3]:
response = {}

In [4]:
def get_item_info(item):
    links = search("green alternatives to "+item)
    titles = []
    urls = []
    for link in links :
        urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
        titles.append(str(link.find('h2').text).replace(" ", " "))
    ts =[]
    us =[]
    i = 0
    for t in titles:
        if any(char.isdigit() for char in t):
            ts.append(t)
            us.append(urls[i])
        i=i+1
    return ts, us

In [5]:
def isrelevant(title, item):
    b = False
    greenwords = ['green', 'eco-friendly', 'sustainable']
    for g in greenwords:
        if g in str.lower(title):
            b = True
    i = ""
    
    '''
    if item[len(item)-1] =='s':
        i =item[:len(item)-1]
    if i in str.lower(title):
        if b:
            return True
    return False
    '''
    return b
  
def get_relevants(titles, urls, item):
    ind = 0
    us = []
    ts = []
    for t in titles:
        if isrelevant(t, item):
            us.append(urls[ind])
            ts.append(titles[ind])
        ind = ind + 1
    return ts, us

In [6]:
t, u = get_item_info("shoes")
titles, urls = get_relevants(t, u, "shoes")

In [7]:
titles

['3 Natural Alternatives to Leather Shoes | Greener Ideal',
 '30 eco-friendly alternatives for things you use every day',
 '7 Sustainable Alternatives to ... - Green Wedding Shoes',
 '5 Sustainable Brands That Make Cool Eco-Friendly Runners ...',
 '6 Eco-Friendly Leather Alternatives for the Cruelty Free ...']

In [8]:
def get_items_from_urls(urls):
    items = []
    for u in urls:
        getRequest = urllib.request.Request(u, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

        urlfile = urllib.request.urlopen(getRequest)
        htmlResult = urlfile.read(2000000)
        urlfile.close()

        soup = BeautifulSoup(htmlResult)
        for h in soup.findAll('h2'):
            b = True
            elem = h
            while(b):
                temp = elem.findChild()
                if(temp==None):
                    b= False
                else:
                    elem = temp
                    
            if len(elem.text)<50:
                items.append(elem.text)
    return items

In [9]:
its = get_items_from_urls(urls)

In [18]:
def filter_items(items):
    ites = []
    b = True
    keywords = ['more', 'support', 'about']
    for i in items:
        if '.' in i:
            ""
        else:
            for k in keywords:
                if k in str.lower(i):
                    b = False
            if b:
                ites.append(i)
            
    return ites

In [23]:
f = filter_items(its)

In [21]:
def get_web_urls(item, name):
    links = search("Buy eco-friendly "+item+" "+name)
    titles = []
    urls = []
    for link in links :
        urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
        titles.append(str(link.find('h2').text).replace(" ", " "))
    
    return titles, urls

In [38]:
t, u = get_search_results(f[0], 'shoes')


In [50]:
links = shop("Buy eco-friendly "+"shoes")

In [49]:
def shop(query):
    address = "http://www.bing.com/shop?q=%s" % (urllib.parse.quote_plus(query))

    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()

    soup = BeautifulSoup(htmlResult)

    results = soup.findAll('div', class_='br-wholeCardClickable br-card br-small br-ocCiteHvrItm')

    return results

In [55]:
links[10]

<div class="br-wholeCardClickable br-card br-small br-ocCiteHvrItm"><a class="br-voidlink" h="ID=commerce,5777.1" href="javascript:void(0)"><div class="cico br-pdMainImg" style="width:256px;height:256px;"><div class="rms_iac" data-alt="Product Image" data-class="rms_img" data-height="256" data-src="/th?id=OP.rpH1cuPZGSFTMg474C474&amp;w=256&amp;h=256&amp;o=5&amp;pid=21.1" data-width="256" style="height:256px;line-height:256px;width:256px;"></div></div></a><div class="br-pdInfo"><div class="pd-price br-standardPrice promoted">$50.00</div><div class="br-vhSlot2" data-priority="2">Free shipping</div><div class="br-sellerBlock"><span><span class="br-sellersCite" tabindex="0">Sanuk</span><span class="br-sellers"><a class="br-adClickOut" h="ID=commerce,5232.1" href="https://www.bing.com/aclick?ld=e8l6H5pkcqXjl0nF8nERTx2jVUCUx6cNGQNgX9ggW2UHX23R75LPSevVBI45oYfPhuckOgQHzqJt-_dxoqhQocfJd8tf4XZbrGdVRo-zKsgu5Lt1So85dqD6FTTSj27bNwiMKyHfa-_K9CdmL5QGZ1EGMqxAw&amp;u=aHR0cCUzYSUyZiUyZmNsaWNrc2VydmUuZGF

In [81]:
def get_shop_data(links):
    shopdata = []
    for l in links:
        data = {}
        try:
            data['image']='https://www.bing.com/'+l.findAll('img')[0]['src']
            data['seller'] = l.findAll('span', class_='br-sellersCite')[0].text
            data['price'] = l.findAll('div', class_='pd-price br-standardPrice promoted')[0].text
            data['item'] = l.findAll('span')[-1]['title']
            data['url'] = l.findAll('a', class_='br-adClickOut')[0]['href']

            shopdata.append(data)
        except:
            print('DIDNT WORK')
            break
    return shopdata
    

In [82]:
get_shop_data(links)

DIDNT WORK


[{'image': 'https://www.bing.com//th?id=OP.d%2fjXkHQMzqlbxw474C474&w=256&h=256&o=5&pid=21.1',
  'seller': 'Sanuk',
  'price': '$50.00',
  'item': "Sanuk Men's Vagabonded Sidewalk Surfers Eco-Friendly Vegan Slip-On Shoes In Black, Size 13",
  'url': 'https://www.bing.com/aclick?ld=e8Tcx4W6__a2WQeh5Av3StwjVUCUwQ4c8l6tGrHKNIxdxoNNWlLfOjKTQz5eB_bgF1K5O8RPDlLI6GlOs9Jqi9mvj4iSeehtq7FNZJV4dFPnlDDC51t9rg7QEsfS2PRF6PZIgSGCNOd0nJuv4q2LG-XBKih3o&u=aHR0cCUzYSUyZiUyZmNsaWNrc2VydmUuZGFydHNlYXJjaC5uZXQlMmZsaW5rJTJmY2xpY2slM2ZsaWQlM2Q5MjcwMDA0Njg4NDQyMTEwNiUyNmRzX3Nfa3dnaWQlM2Q1ODcwMDAwMzg5NzQ2NjU0OSUyNmRzX3NfaW52ZW50b3J5X2ZlZWRfaWQlM2Q5NzcwMDAwMDAwMjUyNTgwNCUyNiUyNmRzX2VfYWRpZCUzZDc2NzU5NzAxMTk4MzkwJTI2ZHNfZV90YXJnZXRfaWQlM2RwbGEtNDU4MDM1OTI4OTUxMDA3NSUyNmRzX2VfcHJvZHVjdF9ncm91cF9pZCUzZDQ1ODAzNTkyODk1MTAwNzUlMjZkc19lX3Byb2R1Y3RfaWQlM2QxMDE1OTI2LURLQi0xMyUyNmRzX2VfcHJvZHVjdF9jb3VudHJ5JTNkVVMlMjZkc19lX3Byb2R1Y3RfbGFuZ3VhZ2UlM2RFTiUyNmRzX2VfcHJvZHVjdF9jaGFubmVsJTNkT25saW5lJTI2ZHNfdXJsX3YlM2QyJTI2ZHNfZGV